In [ ]:
%matplotlib inline
import sys
sys.path.append("../")
from matplotlib import pylab as plt
import numpy as np
from glob import glob
import json
from utils import *
from tqdm import tqdm
import pandas as pd

In [ ]:
!CUDA_VISIBLE_DEVICE=

In [ ]:
codes = [json.loads(s)['code'] for s in open("/home/shido/summarization_java/test.json").readlines()]

In [ ]:
trn_data = read_pickle("../dataset/nl/train.pkl")
vld_data = read_pickle("../dataset/nl/valid.pkl")
tst_data = read_pickle("../dataset/nl/test.pkl")
code_i2w = read_pickle("../dataset/code_i2w.pkl")
code_w2i = read_pickle("../dataset/code_w2i.pkl")
nl_i2w = read_pickle("../dataset/nl_i2w.pkl")
nl_w2i = read_pickle("../dataset/nl_w2i.pkl")

trn_x, trn_y_raw = zip(*trn_data.items())
vld_x, vld_y_raw = zip(*vld_data.items())
tst_x, tst_y_raw = zip(*tst_data.items())

trn_y = [[nl_w2i[t] if t in nl_w2i.keys() else nl_w2i["<UNK>"] for t in l] for l in trn_y_raw]
vld_y = [[nl_w2i[t] if t in nl_w2i.keys() else nl_w2i["<UNK>"] for t in l] for l in vld_y_raw]
tst_y = [[nl_w2i[t] if t in nl_w2i.keys() else nl_w2i["<UNK>"] for t in l] for l in tst_y_raw]

In [ ]:
sorted([len(x) for x in trn_y])[::-1]

In [ ]:
print(len(trn_y))
print(len(vld_y))
print(len(tst_y))

In [ ]:
lengthes = [len(traverse_label(read_pickle(x))) for x in tst_x]

In [ ]:
files = sorted(glob("../models/*/history.json"))
dirs = [x.split("/")[-2] for x in files]
histories = {name: json.load(open(x)) for name, x in zip(dirs, files)}
dirs

In [ ]:
names = [
    "multiway_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1",
    "childsum_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1",
    "deepcom_dim256_embed256_drop0.5_lr0.001_batch64_epochs30_layer1",
    "codenn_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1",
]

In [ ]:
plt.figure(figsize=(15, 10))
# for name, his in [(view, histories[name]) for view, name in zip(["Ours", "Child-Sum", "[Hu+, 18]", "[Iyer+, 16]"], names)]:
for name, his in histories.items():
    if "drop0.5_lr0.001_batch160_epochs50_layer1" in name:
        plt.plot(his["bleu_val"], "-", label=name)
#         plt.plot(his["loss_val"], "-x", label=name + "_valid")
plt.grid()
plt.legend()

In [ ]:
for name, his in histories.items():
    if "drop0.5_lr0.001_batch160_epochs50_layer1" in name:
        print(name, ":", np.mean(his["bleus"]))

In [ ]:
from nltk.translate.gleu_score import sentence_gleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
def get_gleu(true, pred):
    return(sentence_gleu([true], pred))
def get_bleu(true, pred):
    return(sentence_bleu([true], pred, smoothing_function=SmoothingFunction().method4))

In [ ]:
# BLEU
for name, his in histories.items():
    if "drop0.5_lr0.001_batch160_epochs50_layer1" in name:
        trues = histories[name]["trues"]
        preds = histories[name]["preds"]
        gleu = np.mean([get_bleu(x, y) for x, y in zip(trues, preds)])
        print(name, gleu)

In [ ]:
# GLEU
for name, his in histories.items():
    if "drop0.5_lr0.001_batch160_epochs50_layer1" in name:
        trues = histories[name]["trues"]
        preds = histories[name]["preds"]
        gleu = np.mean([get_gleu(x, y) for x, y in zip(trues, preds)])
        print(name, gleu)

In [ ]:
codes = [codes[int(i)] for i in histories[names[0]]["numbers"]]

In [ ]:
df_data = [[" ".join(x) for x in histories[name]["preds"]] for name in names] + [[" ".join(x) for x in histories[names[0]]["trues"]]]
df_data += [histories[name]["bleus"] for name in names] + [codes]
df_index = ["PREDICTION " + name for name in names] + ["GROUND TRUTH"]
df_index += ["BLEU-4 " + name for name in names] + ["SOURCE CODE"]
df = pd.DataFrame(data=df_data, index=df_index).T

In [ ]:
df.to_csv("for_hitachi_lab.csv")

In [ ]:
df.get(["SOURCE CODE", "GROUND TRUTH"]).head()

In [ ]:
shido = histories["multiway_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1"]["bleus"]
np.mean(shido)

In [ ]:
child = histories["childsum_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1"]["bleus"]
np.mean(child)

In [ ]:
dif = np.array(shido) - np.array(child)

In [ ]:
index = np.argsort(dif)[::-1]

In [ ]:
trues = histories["childsumlstm_1layer"]["trues"]

In [ ]:
childsum = histories["childsum_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1"]["preds"]

In [ ]:
ours = histories["multiway_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1"]["preds"]

In [ ]:
number = histories["multiway_dim256_embed256_drop0.5_lr0.001_batch128_epochs30_layer1"]["numbers"]

In [ ]:
for i in index[:500]:
    if " ".join(trues[i]) != " ".join(ours[i]):
        print("GT:   ", " ".join(trues[i]))
        print("CSum: ", " ".join(childsum[i]))
        print("Ours: ", " ".join(ours[i]))
        print("Codes:\n" + codes[i])
#         print("Num:  ", number[i])
        print("-" * 100)

In [ ]:
trn_data = read_pickle("../dataset/nl/train.pkl")
vld_data = read_pickle("../dataset/nl/valid.pkl")
tst_data = read_pickle("../dataset/nl/test.pkl")
code_i2w = read_pickle("../dataset/code_i2w.pkl")
code_w2i = read_pickle("../dataset/code_w2i.pkl")
nl_i2w = read_pickle("../dataset/nl_i2w.pkl")
nl_w2i = read_pickle("../dataset/nl_w2i.pkl")

In [ ]:
trn_data

In [ ]:
xx = [traverse_label(read_pickle(t)) for t in tst_x]

In [ ]:
x = [" ".join([str(code_w2i[w]) for w in t]) + "\n" for t in xx]

In [ ]:
y = [" ".join([str(w) for w in t[1:-1]]) + "\n" for t in tst_y]

In [ ]:
xy = [xw + "\t" + yw for xw, yw in zip(x, y)]

In [ ]:
y[0]

In [ ]:
open("x.tst", "w").writelines(x)

In [ ]:
len(x) / 32

In [ ]:
y[0]

In [ ]:
a = open("x.tst", "r").read()

In [ ]:
a.split("\n")[-2]